In [1]:
!pip install rank_bm25

In [28]:
import json
from rank_bm25 import BM25Okapi

file = "/content/tamu_organizations.json"
with open(file, "r") as f:
  data = json.load(f)

In [29]:
corpus = [item["description"] for item in data]
tokenized_corpus = [doc.lower().split() for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [30]:
query = "indian"
tokenized_query = query.lower().split()
top_n = bm25.get_top_n(tokenized_query, data, n=10) # priority with the name

In [35]:
doc_scores = bm25.get_scores(tokenized_query)

In [32]:
for item in top_n:
    print(f"Organization: {item['name']}")
    print(f"Description: {item['description']}")
    print("-" * 50)

Organization: Society for Promotion of Indian Classical Music & Culture Amongst Youth
Description: The primary objective of this organization shall be to promote Indian classical music and culture amongst the Indian and non-Indian youth in this country. We host Indian Classical Concerts and Student performances
--------------------------------------------------
Organization: South Indian Student Association
Description: The aims of the South Indian Student Association at TAMU are to teach students of all backgrounds about the rich culture and history of South India to promote the representation of mainstream pop culture from south India while fostering a sense of community. South Indian students and those interested in South Indian culture are all welcome, as this is a place providing space to express and celebrate rich heritage.
--------------------------------------------------
Organization: TAMU Sahithya
Description: The mission on TAMU Sahithya is to spread awareness of Indian cult

Colbert Neural reranker

In [ ]:
#https://www.pondhouse-data.com/blog/advanced-rag-colbert-reranker

In [36]:
!pip install "rerankers[transformers]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [37]:
from rerankers import Reranker

# ranker = Reranker('mixedbread-ai/mxbai-rerank-large-v1', model_type='cross-encoder')
ranker = Reranker("colbert")

Loading default colbert model for language en
Default Model: colbert-ir/colbertv2.0
Loading ColBERTRanker model colbert-ir/colbertv2.0 (this message can be suppressed by setting verbose=0)
No device set
Using device cpu
No dtype set
Using dtype torch.float32
Loading model colbert-ir/colbertv2.0, this might take a while...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Linear Dim set to: 128 for downcasting


In [41]:
results = ranker.rank(query=query, docs=[top_n[0]["description"], top_n[1]["description"]])
for result in results:
    print(result)

Result(document=Document(text='The primary objective of this organization shall be to promote Indian classical music and culture amongst the Indian and non-Indian youth in this country. We host Indian Classical Concerts and Student performances', doc_id=0, metadata={}, document_type='text', image_path=None, base64=None), score=1.0283818244934082, rank=1)
Result(document=Document(text='The aims of the South Indian Student Association at TAMU are to teach students of all backgrounds about the rich culture and history of South India to promote the representation of mainstream pop culture from south India while fostering a sense of community. South Indian students and those interested in South Indian culture are all welcome, as this is a place providing space to express and celebrate rich heritage.', doc_id=1, metadata={}, document_type='text', image_path=None, base64=None), score=0.8582708835601807, rank=2)
